In [10]:
import sys
sys.path.insert(0, '../') 

In [11]:
import rdflib
import wlkernel
from pprint import pprint
from collections import Counter

In [12]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [13]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
instances = [ (t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) ) ]
instances = [(x, _) for (x, _) in instances if x != rdflib.term.URIRef('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1959instance')]
assert len(instances) == len(set(instances))
print(len(instances))

177


In [14]:
instances_id = [x[0] for x in instances]
instances_class = [x[1] for x in instances]
# pprint(Counter(instances_class).most_common())
instances_id = [str(x) for x in instances_id]
instances_class = [str(x) for x in instances_class]

In [15]:
employs_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#employs')
head_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#head')
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')

#predicates_to_remove = [employs_uri, head_uri, member_uri, affiliation_uri]
predicates_to_remove = [employs_uri, affiliation_uri]

for predicate in predicates_to_remove:
    rdf_graph.remove( (None, predicate, None) )

In [16]:
triples = ((str(s), str(p), str(o)) for s, p, o in rdf_graph)
g = wlkernel.WLRDFGraph(triples=triples, instances=instances_id, max_depth=2)

In [17]:
kernel_matrix = wlkernel.wlrdf_kernel_matrix(graph=g, instances=instances_id, iterations=0)

In [18]:
# import pandas as pd
# X_df = pd.DataFrame({x: y for x, y in list(enumerate(X))})

In [19]:
X = kernel_matrix
y = instances_class

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)

In [22]:
from sklearn import svm
svm_clf = svm.SVC(kernel='precomputed')
svm_clf.fit(X_train, y_train)

ValueError: X.shape[0] should be equal to X.shape[1]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_trees = 200
rf_clf = RandomForestClassifier(n_estimators=n_trees, max_depth=3)
rf_clf.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

In [ ]:
y_pred_train_rf = rf_clf.predict(X_train)
y_pred_test_rf = rf_clf.predict(X_test)

In [ ]:
y_pred_train_tree = tree_clf.predict(X_train)
y_pred_test_tree = tree_clf.predict(X_test)

In [ ]:
y_pred_train_svm = classifier.predict(X_train)
y_pred_test_svm = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(f'Random Forest - {n_trees} trees:')
print('Train accuracy: ', accuracy_score(y_train, y_pred_train_rf))
print('Test accuracy: ', accuracy_score(y_test, y_pred_test_rf))
print('Decision Tree:')
print('Train accuracy: ', accuracy_score(y_train, y_pred_train_tree))
print('Test accuracy: ', accuracy_score(y_test, y_pred_test_tree))
print('SVM:')
print('Train accuracy: ', accuracy_score(y_train, y_pred_train_svm))
print('Test accuracy: ', accuracy_score(y_test, y_pred_test_svm))